In [1]:
import torch
import torch.nn as nn
import altair as alt
import pandas as pd
import numpy as np
import warnings
import tokenizer
from pathlib import Path
warnings.filterwarnings("ignore")

In [2]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
torch.cuda.empty_cache()


Using device: cuda


In [4]:
from train import DataSetLoader
from config import get_config
from transformer import Transformer
from transformer import TransformerBuilder

D:\Github\SUMMIT\src
=== SUMMIT Training Process ===

Base directory for model-related data: train\opus_books
Checkpoint directory: train\opus_books\checkpoints
Checking devices...
Device for training: cuda
Random seed: 69420
Loading dataset...
Loading raw dataset...
Creating tokenizers...
Looking for tokenizer file: D:\Github\SUMMIT\src\train\opus_books\tokenize\de.json
Found existing tokenizer file, reusing it!
Looking for tokenizer file: D:\Github\SUMMIT\src\train\opus_books\tokenize\en.json
Found existing tokenizer file, reusing it!
Finding longest items...
Longest items found: de: 479, en: 466
Splitting dataset...
Maximum token length found: 500
Train dataset size: 41173
Validation dataset size: 5146
Test dataset size: 5148

Example data entry: {'to_encoder': tensor([    1,  1167,    33,   483,    37,     0,     4,  1221, 17476,    20,
           64,   225,   761,    43,    16,  2121,    21,    12,    20,    54,
        11637,   316,     5,     2,     3,     3,     3,     3,     3

In [5]:
config = get_config()
train_dataloader, validation_dataloader, test_dataloader, vocab_source, vocab_target = DataSetLoader.get_dataset(config)

Loading raw dataset...
Creating tokenizers...
Looking for tokenizer file: D:\Github\SUMMIT\src\train\opus_books\tokenize\de.json
Found existing tokenizer file, reusing it!
Looking for tokenizer file: D:\Github\SUMMIT\src\train\opus_books\tokenize\en.json
Found existing tokenizer file, reusing it!
Finding longest items...
Longest items found: de: 479, en: 466
Splitting dataset...


In [11]:
model = TransformerBuilder.build_transformer(self.tokenizer_source.get_vocab_size(), self.tokenizer_target.get_vocab_size(), self.max_tokens, self.max_tokens, False, True, self.config["MODEL_DIMENSIONS"], self.config["NUM_ENCODER_BLOCKS"], self.config["NUM_HEADS"], self.config["DROPOUT"]).to(self.device)

NameError: name 'self' is not defined

In [8]:
config = get_config()
file_path = str(Path('.').parent.resolve() / config["TRAIN_DIRECTORY"] / config["datasource"] / config["CHECKPOINT_DIRECTORY"] / config["model_name"])
print(file_path)


D:\Github\SUMMIT\src\train\opus_books\checkpoints\00


In [15]:
#Load pretrained weights
state = torch.load(file_path)

In [ ]:
model.load_state_dict(state['model_state_dict'])